In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [18]:
conn1v = sqlite3.connect('database.sqlite')
pd.read_sql("SELECT * FROM sqlite_master ", conn1v)    

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,index,sqlite_autoindex_Player_1,Player,15,None
4,index,sqlite_autoindex_Player_2,Player,17,None
5,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
6,index,sqlite_autoindex_Match_1,Match,19,None
7,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
8,index,sqlite_autoindex_League_1,League,25,None
9,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...


In [37]:
df = pd.read_sql("SELECT * FROM Match ", conn1v)
# df.to_sql('match', conn, if_exists='replace',index=False)
df.column

AttributeError: 'DataFrame' object has no attribute 'column'

In [3]:
rsq("""SELECT 
    -- Select the id, country name, season, home, and away goals
    m.id,
    c.name AS country,
    m.season,
    m.home_goal,
    m.away_goal,
    -- Use a window to include the aggregate average in each row
    AVG(m.home_goal + m.away_goal) OVER() AS overall_avg
FROM match AS m
LEFT JOIN country AS c ON m.country_id = c.id;""")

,id,country,season,home_goal,away_goal,overall_avg
0,8727,Germany,2011/2012,3,1.0,2.858473
1,8728,Germany,2011/2012,2,0.0,2.858473
2,8729,Germany,2011/2012,2,1.0,2.858473
3,8730,Germany,2011/2012,2,0.0,2.858473
4,8731,Germany,2011/2012,3,0.0,2.858473
...,...,...,...,...,...,...
1608,14224,Netherlands,2011/2012,6,4.0,2.858473
1609,23444,Spain,2013/2014,7,3.0,2.858473
1610,24016,Spain,2014/2015,9,1.0,2.858473
1611,24114,Spain,2014/2015,7,3.0,2.858473


In [4]:
rsq("""SELECT 
    -- Select the league name and average goals scored
    l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank each league according to the average goals
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY league_rank;""")

,league,avg_goals,league_rank
0,Italy Serie A,2.436709,1
1,Spain LIGA BBVA,2.763158,2
2,Germany 1. Bundesliga,2.805825,3
3,England Premier League,10.000000,4
4,Netherlands Eredivisie,10.000000,4


In [5]:
rsq("""SELECT 
    -- Select the league name and average goals scored
    l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank leagues in descending order by average goals
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal) DESC) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY league_rank;""")

,league,avg_goals,league_rank
0,Netherlands Eredivisie,10.000000,1
1,England Premier League,10.000000,1
2,Germany 1. Bundesliga,2.805825,3
3,Spain LIGA BBVA,2.763158,4
4,Italy Serie A,2.436709,5


In [7]:
rsq("""SELECT 
    date,
    season,
    home_goal,
    away_goal,
    CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
    -- Calculate the average goals scored partitioned by season
    AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
    AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM match
-- Filter the data set for Legia Warszawa matches only
WHERE 
    hometeam_id = 8673 
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;""")

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-09-14,2013/2014,3,5.0,away,1.793103,1.275862
1,2014-09-13,2014/2015,4,3.0,home,1.769231,1.560000
2,2013-07-20,2013/2014,5,1.0,home,1.793103,1.275862
3,2013-06-02,2012/2013,5,0.0,home,1.869565,1.478261
4,2013-02-23,2012/2013,3,2.0,away,1.869565,1.478261
...,...,...,...,...,...,...,...
95,2014-02-14,2013/2014,1,0.0,home,1.793103,1.275862
96,2014-03-29,2013/2014,1,0.0,home,1.793103,1.275862
97,2013-08-24,2013/2014,0,1.0,home,1.793103,1.275862
98,2013-08-31,2013/2014,0,1.0,away,1.793103,1.275862


In [10]:
rsq("""SELECT 
    date,
    season,
    home_goal,
    away_goal,
    CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
    -- Calculate average goals partitioned by season and month
    AVG(home_goal) OVER(PARTITION BY season, 
            cast(strftime('%m', date) as int)) AS season_mo_home,
    AVG(away_goal) OVER(PARTITION BY season, 
            cast(strftime('%m', date) as int)) AS season_mo_away
--    # AVG(home_goal) OVER(PARTITION BY season, 
--    #      	EXTRACT(MONTH FROM date)) AS season_mo_home,
--    # AVG(away_goal) OVER(PARTITION BY season, 
--    #         EXTRACT(MONTH FROM date)) AS season_mo_away
FROM match
WHERE 
    hometeam_id = 8673 
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;""")

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-09-14,2013/2014,3,5.0,away,2.250000,2.500000
1,2014-09-13,2014/2015,4,3.0,home,2.000000,2.666667
2,2013-07-20,2013/2014,5,1.0,home,2.500000,2.000000
3,2013-02-23,2012/2013,3,2.0,away,3.000000,2.000000
4,2013-06-02,2012/2013,5,0.0,home,5.000000,0.000000
...,...,...,...,...,...,...,...
95,2013-08-24,2013/2014,0,1.0,home,1.500000,0.750000
96,2013-08-31,2013/2014,0,1.0,away,1.500000,0.750000
97,2013-11-10,2013/2014,0,1.0,away,1.666667,0.666667
98,2013-12-01,2013/2014,1,0.0,away,2.250000,0.250000


In [12]:
rsq("""SELECT 
    date,
    home_goal,
    away_goal,
    -- Create a running total and running average of home goals
    SUM(home_goal) OVER(ORDER BY date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
    AVG(home_goal) OVER(ORDER BY date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg
FROM match
WHERE 
    hometeam_id = 9908 
    AND season = '2011/2012';""")

,date,home_goal,away_goal,running_total,running_avg
0,2011-08-14,2,2.0,2,2.000000
1,2011-08-27,3,1.0,5,2.500000
2,2011-09-18,2,2.0,7,2.333333
3,2011-10-01,3,0.0,10,2.500000
4,2011-10-22,1,4.0,11,2.200000
5,2011-11-06,6,4.0,17,2.833333
6,2011-11-06,6,4.0,23,3.285714
7,2011-12-04,2,6.0,25,3.125000
8,2011-12-11,2,2.0,27,3.000000
9,2012-01-22,1,1.0,28,2.800000


In [14]:
rsq("""SELECT 
    -- Select the date, home goal, and away goals
    date,
    home_goal,
    away_goal,
    -- Create a running total and running average of home goals
    SUM(home_goal) OVER(ORDER BY date DESC
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,
    AVG(home_goal) OVER(ORDER BY date DESC
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg
FROM match
WHERE 
    awayteam_id = 9908 
    AND season = '2011/2012';""")

,date,home_goal,away_goal,running_total,running_avg
0,2012-05-06,1,3.0,25,1.470588
1,2012-04-21,0,2.0,24,1.500000
2,2012-04-12,3,0.0,24,1.600000
3,2012-03-25,3,1.0,21,1.500000
4,2012-03-11,1,1.0,18,1.384615
5,2012-02-26,1,0.0,17,1.416667
6,2012-02-05,0,2.0,16,1.454545
7,2012-01-28,2,0.0,16,1.600000
8,2011-12-17,1,0.0,14,1.555556
9,2011-11-25,2,0.0,13,1.625000


In [28]:
rsq("""SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
         WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
         ELSE 'Tie' END AS outcome
FROM match AS m
-- Left join team on the home team ID and team API id
LEFT JOIN team AS t 
ON m.hometeam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the home team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';""")

,id,team_long_name,outcome


In [29]:
rsq("""SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
         WHEN m.home_goal < m.away_goal THEN 'MU Win' 
         ELSE 'Tie' END AS outcome
-- Join team table to the match table
FROM match AS m
LEFT JOIN team AS t 
ON m.awayteam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the away team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';""")

,id,team_long_name,outcome


In [30]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select team names, the date and goals
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015'
      AND (home.team_long_name = 'Manchester United' 
           OR away.team_long_name = 'Manchester United');""")

,date,home_team,away_team,home_goal,away_goal


In [34]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN AWAY ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));""")

,date,home_team,away_team,home_goal,away_goal,match_rank


In [38]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN AWAY ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));""")

,date,home_team,away_team,home_goal,away_goal,match_rank
